In [65]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

## TASK 0.1 

In [67]:
#Setup TensorBoard Writer
writer = SummaryWriter()

In [69]:
#Download and prepare CIFAR-10 dataset 

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='.data',train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='data',train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [70]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [73]:
# Net 1 w/ leaky ReLU
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.leaky_relu(self.conv1(x)))
        x = self.pool(F.leaky_relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Net 2 w/ tanh
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.tanh(self.conv1(x)))
        x = self.pool(F.tanh(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

net1 = Net1()       
net2 = Net2()   

In [75]:
criterion = nn.CrossEntropyLoss()
optimizer1 = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer2 = optim.Adam(net.parameters(), lr=0.001)

In [77]:
# Training function
def train_model(model, optimizer, name):
    for epoch in range(2):  # Train for 2 epochs
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Log loss to TensorBoard
            if i % 100 == 99:
                writer.add_scalar(f"Loss/{name}", running_loss / 100, epoch * len(trainloader) + i)
                running_loss = 0.0

    print(f'Finished Training {name}')

# Train both models
train_model(net1, optimizer1, "Net1_LeakyReLU, SGD")
train_model(net1, optimizer2, "Net1_LeakyReLU, Adam")
train_model(net2, optimizer1, "Net2_Tanh, SGD")
train_model(net2, optimizer2, "Net2_Tanh, Adam")

# Close TensorBoard writer
writer.close()

Finished Training Net1_LeakyReLU, SGD
Finished Training Net1_LeakyReLU, Adam
Finished Training Net2_Tanh, SGD
Finished Training Net2_Tanh, Adam
